In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: C:\Users\MAFC4709\Python_work\RR_model_ECL_parallel
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000029A74F7DC00>, origin='C:\\Users\\MAFC4709\\Python_work\\RR_model_ECL_parallel\\src\\__init__.py', submodule_search_locations=['C:\\Users\\MAFC4709\\Python_work\\RR_model_ECL_parallel\\src'])


In [ ]:
# ===========================================================
# 📘 Roll Rate  - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob

from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)


# --- 2️⃣ Load dữ liệu thực tế ---
#sql_path = r"C:/Users/MAFC4709/Python_work/RR_model/sql/uns2.sql"

#df = load_data(sql_path)
df = load_data()
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])





✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
📦 Loading Parquet from: C:\Users\MAFC4709\Python_work\RR_model_ECL_parallel\data\parquet
✅ Loaded 35,260,160 rows via pyarrow.dataset from C:\Users\MAFC4709\Python_work\RR_model_ECL_parallel\data\parquet


In [ ]:
import sys
sys.path.append("src")

# Reload module khi sửa file
import importlib
import src.rollrate.ead_utils as ead_mod
importlib.reload(ead_mod)

# Import hàm EAD
from src.rollrate.ead_utils import (
    get_latest_ead,
    get_latest_snapshot,
    prepare_ead_input
)

# Test nhanh
latest_ead = get_latest_ead(df)
latest_ead.head()
df_current_raw = latest_ead[~latest_ead[CFG["state"]].isin(["PREPAY", "SOLDOUT"])]

In [5]:
# ============================================================
# 0. IMPORT & SETUP
#    - Mục tiêu: Chuẩn bị module & config dùng trong toàn pipeline
#    - Đầu vào: các file .py đã tồn tại trong src/
#    - Đầu ra: alias tiện dùng: trans_mod, calib_mod, pd_fwd, CFG
#    - Đóng góp: Chuẩn hoá môi trường, tránh import rời rạc
# ============================================================

import pandas as pd
import numpy as np
import importlib

from src.config import CFG
from src.rollrate import transition as trans_mod
from src.rollrate import calibrate_markov_to_real_default as calib_mod
from src.rollrate import pd_forward as pd_fwd

# Reload để chắc chắn đang dùng version mới nhất trong src/
importlib.reload(trans_mod)
importlib.reload(calib_mod)
importlib.reload(pd_fwd)

loan  = CFG["loan"]
state = CFG["state"]
mob   = CFG["mob"]
ead   = CFG["ead"]
cutoff= CFG["cutoff"]


# ============================================================
# 1. CHUẨN BỊ DỮ LIỆU LIFECYCLE CHO TRANSITION & CALIBRATION
#    - Mục tiêu: Lấy panel lifecycle đủ sạch để:
#        + build transition Markov
#        + calibrate k-factor so với default thực tế
#    - Đầu vào: df (lifecycle đầy đủ nhiều cutoff)
#    - Đầu ra:
#        + df_trans: dùng cho compute_transition_by_mob (có thể full, lọc theo product)
#        + df_calib: lifecycle đã thu gọn (18M gần nhất) để chạy calibration tránh MemoryError
#    - Đóng góp:
#        + Giảm kích thước phù hợp RAM
#        + Tách rõ tập dùng cho transition vs tập dùng cho calibration
# ============================================================

# # Giả định: df đã có sẵn (lifecycle panel)
# df[cutoff] = pd.to_datetime(df[cutoff])

# # 1.1. Lọc theo product nếu bạn chỉ làm cho CDLPIL
#df_trans = df[df["PRODUCT_TYPE"] == "CDLPIL"].copy()

# # 1.2. Tạo df_calib = chỉ lấy 18 tháng gần nhất để calibration (giảm tải)
# max_cutoff = df_trans[cutoff].max()
# df_calib = df_trans[df_trans[cutoff] >= (max_cutoff - pd.DateOffset(months=24))].copy()
df_calib = df
# # 1.3. Xoá cột DEFAULT_DATE cũ nếu có, tránh lỗi MergeError khi chạy lại nhiều lần
# drop_cols = [c for c in df_calib.columns if c.startswith("DEFAULT_DATE")]
# if drop_cols:
#     print("💥 Removing old DEFAULT_DATE columns from df_calib:", drop_cols)
#     df_calib = df_calib.drop(columns=drop_cols)

# print("👉 Số dòng df_trans (transition):", len(df_trans))
# print("👉 Số dòng df_calib (calibration):", len(df_calib))


# ============================================================





In [ ]:
# 2. TÍNH TRANSITION MARKOV THEO MOB
#    - Mục tiêu: Xây ma trận P(mob, product, score) từ lifecycle
#    - Đầu vào: df_trans
#    - Đầu ra:
#        + matrices_by_mob: dict[product][mob][score]["P"]
#        + parent_fallback: ma trận fallback theo (product, score)
#    - Đóng góp:
#        + Đây là "engine" Markov gốc để forecast PD
# ============================================================

# print("\n=== STEP 2: BUILD TRANSITION MATRICES ===")
# matrices_by_mob, parent_fallback = trans_mod.compute_transition_by_mob(df_trans)

In [6]:
from src.rollrate.transition import load_transitions, save_transitions

In [7]:
#save_transitions(matrices_by_mob, parent_fallback)  ## lưu  lại để không phải mất công chạy
matrices_by_mob, parent_fallback = load_transitions() ## load lại để không phải mất công chạy


✔ Loaded matrices_by_mob & parent_fallback.


In [8]:

# ============================================================
# 3. CALIBRATION MARKOV → REAL DEFAULT (k-factor)
#    - Mục tiêu:
#        + So sánh PD_12M Markov vs default thực tế trên cùng chuỗi thời gian
#        + Ước lượng k-factor theo segment (product, score)
#        + Scale lại ma trận P cho khớp thực tế
#    - Đầu vào:
#        + df_calib: lifecycle 18M gần nhất
#        + matrices_by_mob, parent_fallback
#    - Đầu ra:
#        + calib_matrices_by_mob, calib_parent_fallback: ma trận P đã hiệu chỉnh k
#        + calib_table: bảng summary k-factor theo segment
#    - Đóng góp:
#        + Đưa Markov về level thực tế (tránh under/over-estimate PD)
# ============================================================

print("\n=== STEP 3: CALIBRATE MARKOV TO REAL DEFAULT ===")
calib_matrices_by_mob, calib_parent_fallback, calib_table = calib_mod.calibrate_markov_to_real_default(
    df_lifecycle=df_calib,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    horizon_months=12,
    min_ead_segment=1e4,
    k_floor=0.4,
    k_cap=3.0,
)

print("✅ Calibration DONE. Một vài dòng calib_table:")
display(calib_table.head())





=== STEP 3: CALIBRATE MARKOV TO REAL DEFAULT ===
✅ Calibration DONE. Một vài dòng calib_table:


,SEG_PRODUCT,SEG_SCORE,TOTAL_EAD,REALIZED_PD_12M,PD_12M_MARKOV_AVG,K_FACTOR,NOTE
0,CDLPIL,500-,9.914267e+06,0.375193,0.382051,0.982048,"calibrated (k_raw=0.982, clipped=[0.4,3.0])"
1,CDLPIL,550-,1.112094e+07,0.257591,0.254961,1.010315,"calibrated (k_raw=1.010, clipped=[0.4,3.0])"
2,CDLPIL,600-,1.244621e+07,0.184442,0.174391,1.057638,"calibrated (k_raw=1.058, clipped=[0.4,3.0])"
3,CDLPIL,650+,1.203303e+07,0.066031,0.056927,1.159920,"calibrated (k_raw=1.160, clipped=[0.4,3.0])"
4,CDLPIL,650-,1.043796e+07,0.124179,0.111985,1.108881,"calibrated (k_raw=1.109, clipped=[0.4,3.0])"


In [9]:
# ============================================================
# 4. CHUẨN BỊ SNAPSHOT HIỆN TẠI ĐỂ TÍNH PD & ECL
#    - Mục tiêu: Có snapshot dư nợ hiện tại (df_current_raw) sạch, không SOLDOUT
#    - Đầu vào:
#        + df_current_raw: đã lấy từ get_latest_ead / logic riêng của bạn
#        + (option) danh sách hợp đồng SOLDOUT nếu muốn loại thêm
#    - Đầu ra:
#        + df_current: bản copy để tính PD, LGD, ECL (không đụng vào raw)
#    - Đóng góp:
#        + Tách hẳn world "history" (lifecycle) và "now" (snapshot)
# ============================================================
df_soldout = pd.read_csv("C:/Users/MAFC4709/Python_work/RR_model_ECL/soldout_list.csv") 
print("\n=== STEP 4: PREPARE CURRENT SNAPSHOT ===")

# Giả định: df_current_raw đã có sẵn, gồm tối thiểu:
#   AGREEMENT_ID, CUTOFF_DATE, PRINCIPLE_OUTSTANDING, MOB, STATE_MODEL, PRODUCT_TYPE, (RISK_SCORE?)
df_current = df_current_raw.copy()

# Nếu bạn có 1 dataframe df_soldout_ids chỉ gồm AGREEMENT_ID cần loại:
df_current = df_current[~df_current[loan].isin(df_soldout[loan])].copy()
# Ở đây tạm comment:
# df_current = df_current[~df_current[loan].isin(df_soldout_ids[loan])].copy()

print("👉 Số dòng df_current (snapshot EAD):", len(df_current))





=== STEP 4: PREPARE CURRENT SNAPSHOT ===
👉 Số dòng df_current (snapshot EAD): 553003


In [10]:
# ============================================================
# 5. TÍNH PD_12M CHO SNAPSHOT HIỆN TẠI (FORWARD PD)
#    - Mục tiêu: Gán PD_12M Markov đã calibrate cho từng khoản vay hiện tại
#    - Đầu vào:
#        + df_current
#        + calib_matrices_by_mob, calib_parent_fallback
#    - Đầu ra:
#        + df_pd: [AGREEMENT_ID, PD_12M]
#        + df_current cập nhật thêm cột PD_12M
#    - Đóng góp:
#        + Đây là PD đầu vào để tính ECL_12M (Stage 1)
# ============================================================

print("\n=== STEP 5: FORWARD PD_12M FOR CURRENT SNAPSHOT ===")

df_pd = pd_fwd.compute_forward_pd(
    df_current=df_current,
    matrices_by_mob=calib_matrices_by_mob,
    parent_fallback=calib_parent_fallback,
    horizon=12,
)

df_current = df_current.merge(df_pd[[loan, "PD_12M"]], on=loan, how="left")
df_current["PD_12M"].fillna(0.0, inplace=True)

print("Mô tả nhanh PD_12M hiện tại:")
display(df_current["PD_12M"].describe())





=== STEP 5: FORWARD PD_12M FOR CURRENT SNAPSHOT ===
Mô tả nhanh PD_12M hiện tại:


C:\Users\MAFC4709\AppData\Local\Temp\ipykernel_21464\1926904216.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_current["PD_12M"].fillna(0.0, inplace=True)


count    553003.000000
mean          0.295012
std           0.332691
min           0.000000
25%           0.082097
50%           0.136611
75%           0.328600
max           1.000000
Name: PD_12M, dtype: float64

## check PD

In [12]:
df_trans = df

In [ ]:
from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.pd_forward import compute_forward_pd
from src.rollrate.debug_pd_forward import analyze_pd_by_state_and_mob, debug_single_state

# Giả sử df_trans là lịch sử panel
matrices_by_mob, parent_fallback = compute_transition_by_mob(df_trans)

# ✅ 1) Soi PD_12M cho mọi state & MOB (0–12) của 1 segment
df_grid = analyze_pd_by_state_and_mob(
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    product="SALPIL",
    score="1",
    mob_min=0,
    mob_max=12,
    horizon=12,
)
display(df_grid)

# ✅ 2) Soi riêng DPD0 theo MOB (0–6)
debug_single_state(
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    product="SALPIL",
    score="1",
    state="DPD0",
    mob_min=0,
    mob_max=6,
    horizon=12,
)


In [ ]:
# ============================================================
# 6. MAP LGD (LGD_BASE → LGD_EFF) & TÍNH ECL_12M
#    - Mục tiêu:
#        + Gắn LGD theo PRODUCT_SEGMENT & MOB_BUCKET
#        + Tính LGD_EFF thực tế cho từng khoản vay
#        + Tính ECL_12M = PD_12M × LGD_EFF × EAD
#    - Đầu vào:
#        + df_current (có PD_12M, MOB, PRODUCT_TYPE)
#        + lgd_base: bảng LGD bạn đã có (PRODUCT_SEGMENT, MOB_BUCKET, LGD_BASE)
#    - Đầu ra:
#        + df_lgd: bảng đầy đủ để báo cáo (detail)
#    - Đóng góp:
#        + Ghép đủ 3 trụ cột PD – LGD – EAD → ECL_12M
# ============================================================

print("\n=== STEP 6: MAP LGD & COMPUTE ECL_12M ===")

# 6.1. Tính MOB_AT_DEFAULT ~ MOB + 6 (giả định sự kiện default trung bình tại 6/12M)
df_current["MOB_AT_DEFAULT"] = (df_current[mob] + 6).astype(int)
df_current["MOB_AT_DEFAULT"] = df_current["MOB_AT_DEFAULT"].clip(lower=0)

# 6.2. Chuẩn hoá LGD base
lgd = lgd_base.copy()
lgd["MOB_BUCKET_MIN"] = lgd["MOB_BUCKET"].str.split("-").str[0].astype(int)
lgd["MOB_BUCKET_MAX"] = lgd["MOB_BUCKET"].str.split("-").str[1].astype(int)

# 6.3. Merge theo PRODUCT_TYPE
df_lgd = df_current.merge(
    lgd,
    left_on="PRODUCT_TYPE",
    right_on="PRODUCT_SEGMENT",
    how="left"
)

# 6.4. Lọc đúng bucket MOB
mask_bucket = (
    (df_lgd["MOB_AT_DEFAULT"] >= df_lgd["MOB_BUCKET_MIN"]) &
    (df_lgd["MOB_AT_DEFAULT"] <= df_lgd["MOB_BUCKET_MAX"])
)
df_lgd = df_lgd[mask_bucket].copy()

# 6.5. LGD_EFF
df_lgd["LGD_EFF"] = df_lgd["LGD_BASE"].fillna(df_lgd["LGD_BASE"].mean())
df_lgd["LGD_EFF"].fillna(0.5, inplace=True)   # fallback cuối

# 6.6. Tính ECL_12M
df_lgd["ECL_12M"] = df_lgd["PD_12M"] * df_lgd["LGD_EFF"] * df_lgd[ead]
df_lgd["ECL_12M"].fillna(0.0, inplace=True)

print("Một vài dòng df_lgd (detail):")
display(df_lgd[[loan, state, mob, "PRODUCT_TYPE", "PD_12M", "LGD_EFF", ead, "ECL_12M"]].head())




In [ ]:
# ============================================================
# 7. SUMMARY & EXPORT EXCEL REPORT
#    - Mục tiêu:
#        + Tạo các bảng summary phục vụ báo cáo:
#            * Summary tổng
#            * By Product
#            * By MOB
#            * By Score
#            * By State
#            * By Segment (Product×Score)
#        + Xuất 1 file Excel duy nhất: ECL_Report.xlsx
#    - Đầu vào:
#        + df_lgd (detail)
#    - Đầu ra:
#        + summary, summary_by_* (DataFrame)
#        + file Excel
#    - Đóng góp:
#        + Đây là deliverable cuối cùng cho báo cáo ECL Stage 1
# ============================================================

print("\n=== STEP 7: SUMMARY & EXPORT EXCEL ===")

total_ead = df_lgd[ead].sum()
total_ecl = df_lgd["ECL_12M"].sum()
weighted_pd  = (df_lgd["PD_12M"] * df_lgd[ead]).sum() / total_ead
weighted_lgd = (df_lgd["LGD_EFF"] * df_lgd[ead]).sum() / total_ead

summary = pd.DataFrame({
    "TOTAL_EAD": [total_ead],
    "TOTAL_ECL": [total_ecl],
    "WEIGHTED_PD": [weighted_pd],
    "WEIGHTED_LGD": [weighted_lgd],
    "ECL_RATE": [total_ecl / total_ead],
})

# 7.1. Summary theo PRODUCT
report_by_product = (
    df_lgd.groupby("PRODUCT_TYPE")
    .agg(
        TOTAL_EAD=(ead, "sum"),
        TOTAL_ECL=("ECL_12M", "sum"),
        AVG_PD=("PD_12M", "mean"),
        W_PD=lambda g: (g["PD_12M"] * g[ead]).sum() / g[ead].sum(),
        W_LGD=lambda g: (g["LGD_EFF"] * g[ead]).sum() / g[ead].sum(),
    )
)

# 7.2. Summary theo MOB
summary_by_mob = (
    df_lgd.groupby(mob)
    .agg(
        TOTAL_EAD=(ead, "sum"),
        TOTAL_ECL=("ECL_12M", "sum"),
        AVG_PD=("PD_12M", "mean"),
        W_PD=lambda g: (g["PD_12M"] * g[ead]).sum() / g[ead].sum(),
        W_LGD=lambda g: (g["LGD_EFF"] * g[ead]).sum() / g[ead].sum(),
    )
    .reset_index()
)

# 7.3. Summary theo SCORE (nếu có)
score_col = "RISK_SCORE" if "RISK_SCORE" in df_lgd.columns else None
if score_col:
    summary_by_score = (
        df_lgd.groupby(score_col)
        .agg(
            TOTAL_EAD=(ead, "sum"),
            TOTAL_ECL=("ECL_12M", "sum"),
            AVG_PD=("PD_12M", "mean"),
            W_PD=lambda g: (g["PD_12M"] * g[ead]).sum() / g[ead].sum(),
            W_LGD=lambda g: (g["LGD_EFF"] * g[ead]).sum() / g[ead].sum(),
        )
        .reset_index()
    )
else:
    summary_by_score = pd.DataFrame({"NOTE": ["Không có cột RISK_SCORE"]})

# 7.4. Summary theo STATE_MODEL
summary_by_state = (
    df_lgd.groupby(state)
    .agg(
        TOTAL_EAD=(ead, "sum"),
        TOTAL_ECL=("ECL_12M", "sum"),
        AVG_PD=("PD_12M", "mean"),
        W_PD=lambda g: (g["PD_12M"] * g[ead]).sum() / g[ead].sum(),
        W_LGD=lambda g: (g["LGD_EFF"] * g[ead]).sum() / g[ead].sum(),
    )
    .reset_index()
)

# 7.5. Summary theo SEGMENT (PRODUCT × SCORE)
if score_col:
    df_lgd["SEGMENT"] = df_lgd["PRODUCT_TYPE"].astype(str) + "_" + df_lgd[score_col].astype(str)
else:
    df_lgd["SEGMENT"] = df_lgd["PRODUCT_TYPE"]

summary_by_segment = (
    df_lgd.groupby("SEGMENT")
    .agg(
        TOTAL_EAD=(ead, "sum"),
        TOTAL_ECL=("ECL_12M", "sum"),
        AVG_PD=("PD_12M", "mean"),
        W_PD=lambda g: (g["PD_12M"] * g[ead]).sum() / g[ead].sum(),
        W_LGD=lambda g: (g["LGD_EFF"] * g[ead]).sum() / g[ead].sum(),
    )
)

# 7.6. Export Excel
output_file = "ECL_Report.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    df_lgd.to_excel(writer, sheet_name="Detail", index=False)
    summary.to_excel(writer, sheet_name="Summary", index=False)
    report_by_product.to_excel(writer, sheet_name="By_Product")
    summary_by_mob.to_excel(writer, sheet_name="By_MOB", index=False)
    summary_by_score.to_excel(writer, sheet_name="By_Score", index=False)
    summary_by_state.to_excel(writer, sheet_name="By_State", index=False)
    summary_by_segment.to_excel(writer, sheet_name="By_Segment")

print(f"✅ DONE! ECL report exported to: {output_file}")

# Hiển thị nhanh summary
display(summary)